In [1]:
# %pip install pygame

In [2]:
import pygame
import random
import math
import numpy as np

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
global ground

# Society attributes
society_attr = {
  "Digitarian": {
    'energy': 100,
    'area': 16,
    'population': 16,
  },
  "Communo": {
    'energy': 999,
    'area': 16,
    'population': 16,
  },
  "Evolutionist": {
    'energy': 100,
    'area': 40,
    'population': 16,
  },
  "Bio": {
    'energy': 100,
    'area': 16,
    'population': 16,
  },
}



In [4]:
# Initialize Pygame
pygame.init()

# Define constants
WINDOW_WIDTH, WINDOW_HEIGHT = 1000, 700
GRID_SIZE = 20
# grid_width, grid_height = 40, 40
block_size = 12
agent_size = 5
GRID_WIDTH, GRID_HEIGHT = WINDOW_WIDTH // GRID_SIZE, WINDOW_HEIGHT // GRID_SIZE

FPS = 60

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)


In [87]:
# Agent class
class Human(pygame.sprite.Sprite):
    def __init__(self, x, y, society, toxic_resistance, resident_idx):
        super().__init__()
        self.image = pygame.Surface((agent_size, agent_size))
        self.image.fill(WHITE)
        self.rect = self.image.get_rect(center=(x, y))
        self.health = 100
        self.happiness = 50
        self.mutations = 0
        self.toxic_resistance = toxic_resistance
        self.power = 100
        self.society = society
        self.cost = 1    
        match self.society:
            case "Digitarian":
                self.resident_idx = resident_idx
            case "Communo":
                self.cost = 0.5    
            case "Evolutionist":
                self.positiveMutations = 0
        

    def update(self, agents):
        # neighbours = [(-1,-1), (-1,0), (-1,1), (0,1), (1,1), (1,0), (1,-1), (0,-1)]

        for i in range(self.rect.x-GRID_SIZE, self.rect.x+GRID_SIZE+1):
            for j in range(self.rect.y-GRID_SIZE, self.rect.y+GRID_SIZE+1):
                pass


        x = self.rect.x + random.randrange(-GRID_SIZE, GRID_SIZE+1, GRID_SIZE)
        y = self.rect.y + random.randrange(-GRID_SIZE, GRID_SIZE+1, GRID_SIZE)

        if x >= 10 and x <= WINDOW_WIDTH-10:
            self.rect.x = x
        if y >= 10 and y <= WINDOW_HEIGHT-10:
            self.rect.y = y
        

        

    def get_resource(self):
        grid.ground[(self.x - block_size)/GRID_SIZE][(self.x - block_size)/GRID_SIZE].generate_energy()
        society_attr[self.society].energy -= self.cost


    # Digitarian
    def detectToxicity(self):
        for i in range(3):
            for j in range(3):
                if (grid.ground[(self.x - block_size)/GRID_SIZE + i][(self.x - block_size)/GRID_SIZE + j].toxicity > 0)
                    #avoid logic
                    pass
        society_attr[self.society].energy -= self.cost


    
    # Communo
    def share(self):
        agents = grid.ground[(self.x - block_size)/GRID_SIZE + i][(self.x - block_size)/GRID_SIZE + j].agents
        random = agents[random.randint(agents.length -1)]
        health_share = (random.health + self.health)/2
        happiness_share = (random.happiness + self.happiness)/2
        power_share = (random.happiness + self.happiness)/2
        self.health = health_share
        random.health = health_share
        self.happiness = happiness_share
        random.happiness = happiness_share
        self.power = power_share
        random.power = power_share
        society_attr[self.society].energy -= self.cost

    # Bio
    def fertilizeTile(self):
        grid.ground[(self.x - block_size)/GRID_SIZE + i][(self.x - block_size)/GRID_SIZE + j].fertility += random.randint(2, 8)
        society_attr[self.society].energy -= self.cost


    # Evolutionist
    def forceMutation(self):
        if(random.random > 0.9):
            self.positiveMutations = self.positiveMutations + 1
        else:
            self.mutations = self.mutations + 1
        society_attr[self.society].energy -= 1


In [88]:
class Grid():
  def __init__(self):
        self.ground = [[None for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)] 

  def update(self):
    pass

In [89]:
# Ground class
class Ground(pygame.sprite.Sprite):
    def __init__(self, x, y, fertility, toxicity, resources, max_citizens, regeneration ,type):
        super().__init__()
        self.type = type
        self.image = pygame.Surface((block_size, block_size))
        # self.image = pygame.Rect((10,10))
        self.image.fill(BLUE)
        self.rect = self.image.get_rect(center=(x, y))
        self.fertility = fertility
        self.toxicity = toxicity
        self.resources = resources
        self.level = 1
        self.max_citizens = max_citizens
        self.regeneration = regeneration
        self.allow_demutate = False
        self.agents = []
        # Define properties based on type

    def update(self):
        pass
    
    def level_up (self):
        if(self.level == 3):
            return
        self.level = self.level + 1

    #Lab

    def decreaseMutation(self, agent):
        if(self.type == "Lab"):
            if(random.random() > 0.88):
                agent.mutations -= 1
        
    
    # City    
    def generate_energy(self, agent):
        basic = random.randint(5,10)
        energyMultiplier = 1
        decreaseFertility = 1
        match self.type:
            case 'City':
                energyMultiplier += 1
            case 'Lab':
                energyMultiplier += 1
            case 'Nuclear':
                match self.level:
                    case 1:
                        energyMultiplier += 3
                    case 2:
                        energyMultiplier += 6
                    case 3:
                        energyMultiplier += 9
            case 'Farm':
                energyMultiplier += 1
                if(self.level == 3):
                    decreaseFertility = 0
        society_attr[agent.type].energy += basic * energyMultiplier
        if(random.random() > 0.85):
            self.fertility = self.fertility - decreaseFertility
    

    def give_happines(self, agent):
        if (self.level >= 2):
            agent.happines += radnom.randint(5,10)
        

In [90]:
# Setup Pygame window
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Agent Systems Simulation")
clock = pygame.time.Clock()

# Sprite group for agents
grid = Grid()
agents = pygame.sprite.Group()
ground = pygame.sprite.Group()


for i in range(0, GRID_WIDTH):
    for j in range(0, GRID_HEIGHT):
        fertility = random.randint(50,100)
        toxicity = random.randint(0,10)
        resources = random.randint(0,5)
        max_citizens = 1
        regeneration = random.random()
        type = 'plain'
        # g = Ground(i * WINDOW_WIDTH/GRID_WIDTH+block_size, j*WINDOW_HEIGHT/GRID_HEIGHT+block_size, fertility, toxicity, resources, max_citizens, regeneration, type)
        
        g = Ground(i*GRID_SIZE+block_size, j*GRID_SIZE+block_size, fertility, toxicity, resources, max_citizens, regeneration, type)

        # j=x i=y
        grid.ground[j][i] = g

        ground.add(g)


# Create agents
for i in range(16):
    x, y = random.randint(0, GRID_WIDTH//2), random.randint(0, GRID_HEIGHT//2)
    # agent = Human(x*20+block_size//2+random.randint(agent_size,agent_size*2), y*20+block_size//2+random.randint(agent_size,agent_size*2), "Digitarian", random.randint(0, 10), i)
    agent = Human(x*GRID_SIZE+block_size, y*GRID_SIZE+block_size, "Digitarian", random.randint(0, 10), i)
    agents.add(agent)
    # print(f'{x} {y} {agent.rect.centerx} {agent.rect.centery}')
    # print(f'{grid.ground[y][x].agents}')
    grid.ground[y][x].agents.append(agent)
    # print(f'{grid.ground[y][x].agents}')

# print(grid.ground)

# for i in range(GRID_WIDTH):
#     for j in range(GRID_HEIGHT): 
#         if grid.ground[j][i].agents != []:
#             print(f'{grid.ground[j][i].rect.centerx} {grid.ground[j][i].agents[0].rect.centerx}')

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
        # Update
                agents.update(agents)
                ground.update()
                # Draw
                window.fill(BLACK)
                ground.draw(window)
                agents.draw(window)

                pygame.display.flip()
                clock.tick(FPS)

pygame.quit()

370 130
330 310
50 190
390 130
450 270
110 90
70 50
270 190
270 250
350 250
190 270
510 110
490 310
70 30
210 290
390 110
350 130
350 290
70 190
370 150
430 250
130 90
70 30
290 190
290 250
370 230
190 290
530 110
490 290
70 30
230 290
390 90
330 130
350 310
70 190
390 130
450 270
150 110
90 30
310 170
290 230
370 210
190 310
530 90
510 270
70 30
210 270
390 110
330 130
350 290
70 210
390 110
450 250
170 130
90 30
330 170
290 250
370 210
210 290
510 110
510 250
50 30
190 290
390 90
330 130
350 290
50 230
410 90
430 270
150 150
110 30
330 170
310 230
350 230
190 290
490 90
530 250
70 10
210 290
370 70
350 110
370 270
50 250
430 70
450 250
130 130
110 10
330 170
290 210
350 250
170 310
470 90
510 250
70 30
210 290
350 70
350 90
370 270
70 270
410 70
430 270
150 150
130 30
350 150
270 190
350 270
170 310
470 70
510 250
90 30
190 290
330 50
370 90
350 270
90 250
430 50
430 290
170 130
130 50
370 130
270 170
370 250
170 290
450 50
490 250
90 50
190 310
310 70
370 90
330 290
70 230
410 50
45